In [3]:
import requests, json, pprint, string, collections, tqdm, networkx as nx, matplotlib.pyplot as plt

In [4]:
# G=nx.generators.diamond_graph()
# nx.draw(G)
# plt.draw()
# plt.show()

In [4]:
API_LINK = "http://api.slowosiec.clarin-pl.eu:80/plwordnet-api/"
SENSE_ID_FOR_SYNSET = "synsets/search?lemma="
SYNSET_ID_template = string.Template("senses/$ID/synset")
GET_SYNSET_GIVEN_ID = string.Template("synsets/$ID/cache")

#############

SENSE_ID = string.Template("senses/search?lemma=$LEMMA&&&&&&&&&")
RELATIONS_FOR_SYNSET_ID = string.Template("synsets/$ID/relations")
GET_SENSE_BASED_ON_SYNSET_ID = string.Template("synsets/$ID/senses")

In [5]:
def get_sense_id(word):
    link = API_LINK + SENSE_ID_FOR_SYNSET + word + "&&&&&&&&&"
    response = requests.get(link).text
    json_content = json.loads(response)
    senses_id = []
    for result in json_content:
        if result['lemma']['word'] == word:
            sense_id = result['id']
            senses_id.append(sense_id)
    return senses_id

def get_synset_id(sense_id):
    link = API_LINK + SYNSET_ID_template.substitute(ID=sense_id)
    response = requests.get(link).text
    json_content = json.loads(response)
    return json_content['id']

def get_synset_given_id(synset_id):
    link = API_LINK + GET_SYNSET_GIVEN_ID.substitute(ID=synset_id)
    response = requests.get(link).text
    json_content = json.loads(response)
    senses = []
    lemmas = []
    for element in range(len(json_content)):
        sense = json_content[element]['synset']['senses']
        for lemma in sense.values():
            lemmas.append(lemma)
    return lemmas

In [6]:
relationRecord = collections.namedtuple(
    'relationRecord', ['synset_from_id', 'synset_to_id', 'relation_type'])

In [7]:
def get_sense_id_based_on_lemma(lemma):
    link = API_LINK + SENSE_ID.substitute(LEMMA=lemma)
    response = requests.get(link).text
    json_content = json.loads(response)
    if len(json_content['content'])>0:
        return json_content['content'][0]['id']
    return None

def get_relations_based_on_synset_id(synset_id):
    link = API_LINK + RELATIONS_FOR_SYNSET_ID.substitute(ID=synset_id)
    response = requests.get(link).text
    json_content = json.loads(response)
    return json_content

def get_relations_of_given_type_based_on_synset_id(synset_id, relation_type):
    json_content = get_relations_based_on_synset_id(synset_id)
    response_len = len(json_content)
    results = []
    for idx in range(response_len):
        relation_name = json_content[idx]['relation']['name']
        if relation_name == relation_type:
            synsetFrom = str(json_content[idx]['synsetFrom']['id'])
            synsetTo = str(json_content[idx]['synsetTo']['id'])
            relationType = str(json_content[idx]['relation']['name'])
            relation = relationRecord(synsetFrom, synsetTo, relationType)
            results.append(relation)
    return results

def get_sense_based_on_synset_id(synset_id):
    link = API_LINK + GET_SENSE_BASED_ON_SYNSET_ID.substitute(ID=synset_id)
    response = requests.get(link).text
    json_content = json.loads(response)
    lemmas = []
    for element in range(len(json_content)):
        lemmas.append(json_content[element]['lemma']['word'])
    return lemmas

In [9]:
# Znajdź wszystkie znaczenia rzeczownika szkoda oraz wymień ich synonimy (jeśli posiadają).
senses_id = get_sense_id("szkoda")
for sense_id in senses_id:
    synset_id = get_synset_id(sense_id)
    lemmas = get_synset_given_id(synset_id)
    for lemma in lemmas:
        pprint.pprint(lemma['lemma']['word'])
    print ("##################################################")

'tęsknić'
'tęsknić'
'szkoda'
'żałować'
'żal'
'opłakiwać'
'płakać'
'doświadczać negatywnego uczucia'
'doświadczać negatywnego uczucia'
'opłakiwać'
'płakać'
'commiseration'
'pity'
'ruth'
'pathos'
'commiseration'
'pity'
'ruth'
'pathos'
##################################################
'żerowisko'
'field'
'pole'
'łan'
'niwa'
'rola'
'pole uprawne'
'szkoda'
'żerowisko'
'field'
'pole'
'łan'
'niwa'
'rola'
'pole uprawne'
##################################################


In [118]:
def get_hiperonims_for_lemma(lemma, relation='hiperonimia'):
    sense_id = get_sense_id_based_on_lemma(lemma)
    synset_id = get_synset_id(sense_id)
    relationRecordResult = get_relations_of_given_type_based_on_synset_id(synset_id, relation)
    results = []
    for idx in range(len(relationRecordResult)):
        results.append((
            get_sense_based_on_synset_id(relationRecordResult[idx].synset_to_id)[0],
             get_sense_based_on_synset_id(relationRecordResult[idx].synset_from_id)[0]))
    return results

In [125]:
def get_hiponims_for_lemma(lemma):
    return get_hiperonims_for_lemma(lemma, relation='hiponimia')

In [128]:
def get_transitive_closure_for_lemma(lemma, func):
    hiperonims = []
    res = get_hiperonims_for_lemma(lemma)
    hiperonims.append(res[0])
    for tmp in hiperonims:
        print (hiperonims)
        insertAnything = False
        word = tmp[1]
        results = func(word)
        for result in results:
            if result not in hiperonims:
                hiperonims.append(result)
                insertAnything = True
        if not insertAnything:
            return hiperonims

In [58]:
print(get_hiperonims_for_lemma("wypadek%20drogowy"))
print(get_hiperonims_for_lemma("wypadek%20komunikacyjny"))

[('wypadek drogowy', 'wypadek komunikacyjny')]
[('wypadek drogowy', 'wypadek komunikacyjny'), ('wypadek komunikacyjny', 'wypadek')]


In [122]:
results = get_transitive_closure_for_lemma("wypadek%20drogowy", get_hiperonims_for_lemma)

In [123]:
results

[('wypadek drogowy', 'wypadek komunikacyjny'),
 ('wypadek komunikacyjny', 'wypadek'),
 ('zawał', 'wypadek'),
 ('tąpnięcie', 'wypadek'),
 ('katastrofa', 'wypadek'),
 ('wykolejenie', 'wypadek'),
 ('zakrztuszenie', 'wypadek'),
 ('katastrofa budowlana', 'wypadek'),
 ('wypadek', 'zdarzenie oceniane negatywnie')]

In [13]:
# Znajdź bezpośrednie hiponimy rzeczownika wypadek1.
sense_id = get_sense_id_based_on_lemma("wypadek")
synset_id = get_synset_id(sense_id)
print ("Wypadek drogowy, synet id: {0}".format(str(synset_id)))
relationRecordResult = get_relations_of_given_type_based_on_synset_id(synset_id, "hiponimia")
senseFrom = get_sense_based_on_synset_id(relationRecordResult[0].synset_from_id)

words = []

for idx in range(len(relationRecordResult)):
    print ("Relation of : {0}, from sense: {1} to sense: {2}".format( 
          relationRecordResult[idx].relation_type, 
          get_sense_based_on_synset_id(relationRecordResult[idx].synset_from_id),
          get_sense_based_on_synset_id(relationRecordResult[idx].synset_to_id)))
    words.append(get_sense_based_on_synset_id(relationRecordResult[idx].synset_from_id))

Wypadek drogowy, synet id: 3982
Relation of : hiponimia, from sense: ['wypadek'] to sense: ['zdarzenie oceniane negatywnie']
Relation of : hiponimia, from sense: ['zawał'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['tąpnięcie'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['katastrofa'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['wykolejenie'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['zakrztuszenie', 'zachłyśnięcie'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['wypadek komunikacyjny'] to sense: ['wypadek']
Relation of : hiponimia, from sense: ['katastrofa budowlana'] to sense: ['wypadek']


In [136]:
res = get_hiponims_for_lemma('wypadek')

In [137]:
print (res)

[('zdarzenie oceniane negatywnie', 'wypadek'), ('wypadek', 'zawał'), ('wypadek', 'tąpnięcie'), ('wypadek', 'katastrofa'), ('wypadek', 'wykolejenie'), ('wypadek', 'zakrztuszenie'), ('wypadek', 'wypadek komunikacyjny'), ('wypadek', 'katastrofa budowlana')]


In [22]:
def get_synset_id_for_lemma(lemma):
    sense_id = get_sense_id_based_on_lemma(lemma)
    synset_id = get_synset_id(sense_id)
    return synset_id

def get_synset_id_for_list_of_lemmas(lemmas):
    lemmas_ids = [(lemma, get_synset_id_for_lemma(lemma)) for lemma in lemmas]
    return lemmas_ids

#szkoda2, strata1, uszczerbek1, 
#szkoda majątkowa1, uszczerbek na zdrowiu1, krzywda1, niesprawiedliwość1, nieszczęście2.
get_synset_id_for_list_of_lemmas(['szkoda', 'strata', 
                                   'uszczerbek', 'szkoda%20majątkowa',
#                                   'uszczerbek%20%na%20zdrowiu', 'krzywda'
                                  'niesprawiedliwość', 'nieszczęście'])

# sense_id = get_sense_id_based_on_lemma('szkoda')
# synset_id = get_synset_id(sense_id)
# relations = get_relations_based_on_synset_id(synset_id)
# pprint.pprint (relations)

KeyError: 'id'